# BTSnoop Log Sniffer

This notebook demonstrates the process of parsing bluetooth packet data from android hci logs.

In [ ]:
import btsnoop.btsnoop.btsnoop as bts
import btsnoop.bt.hci as hci
import btsnoop.bt.hci_uart as hci_uart
import btsnoop.bt.hci_cmd as hci_cmd
import btsnoop.bt.hci_evt as hci_evt
import btsnoop.bt.hci_acl as hci_acl
import btsnoop.bt.hci_sco as hci_sco
import btsnoop.bt.l2cap as l2cap
import btsnoop.bt.att as att
import btsnoop.bt.smp as smp
import binascii
import pandas as pd
from pprint import pprint

records = bts.parse("./bt_logfiles/btsnoop_hci_10022024_1930.log")


# TODO: SQLite storage for known devices

btsnoop capture file version 1, type 1002


In [82]:
rows = []
for record in records:
    row = {}
    seq = record[0]
    packet_length = record[1]
    flags_raw = record[2]
    flags = bts.flags_to_str(flags_raw)
    pkt_src = flags[0].capitalize()
    pkt_dst = flags[1].capitalize()
    pkt_type = flags[2]
    time = record[3].strftime("%b-%d %H:%M:%S.%f")
    pkt_data = record[4]

    direction = bts.flags_to_direction(flags_raw)

    hci_pkt_type, hci_pkt_data = hci_uart.parse(record[4])
    hci = hci_uart.type_to_str(hci_pkt_type)

    row.append({'sequence_number':seq,
         'packet_length':packet_length,
         'pkt_src':pkt_src,
         'pkt_dst':pkt_dst,
         'pkt_type':pkt_type,
         'pkt_data':pkt_data})

    if hci_pkt_type == hci_uart.HCI_CMD:
        opcode, length, data = hci_cmd.parse(hci_pkt_data)
        cmd_evt_l2cap = hci_cmd.cmd_to_str(opcode)
        row.append(['opcode':opcode,
                    'hci_cmd_data':data, 'cmd_evt_l2cap':cmd_evt_l2cap])
    elif hci_pkt_type == hci_uart.HCI_EVT:
        hci_data = hci_evt.parse(hci_pkt_data)
        evtcode, data = hci_data[0], hci_data[-1]
        cmd_evt_l2cap = hci_evt.evt_to_str(evtcode)
        row.extend([evtcode, data, cmd_evt_l2cap])
    elif hci_pkt_type == hci_uart.SCO_DATA:
        handle, ps, length, data = hci_sco.parse(hci_pkt_data)
        row.extend([handle, ps, data])
        # l2cap_length, l2cap_cid, l2cap_data = l2cap.parse(hci_data[2], data)

        # data = binascii.hexlify(data)
        # data = len(data) > 30 and data[:30] + "..." or data
        # print(handle, ps, length, data)

        raise Exception('DEBUG: SCO Data!')
    elif hci_pkt_type == hci_uart.ACL_DATA:
        hci_data = hci_acl.parse(hci_pkt_data)
        l2cap_length, l2cap_cid, l2cap_data = l2cap.parse(hci_data[2], hci_data[4])

        if l2cap_cid == l2cap.L2CAP_CID_LE_ATT:
            att_opcode, att_data = att.parse(l2cap_data)
            cmd_evt_l2cap = att.opcode_to_str(att_opcode, att_data)
            data = att_data

        elif l2cap_cid == l2cap.L2CAP_CID_LE_SMP:
            smp_code, smp_data = smp.parse(l2cap_data)
            cmd_evt_l2cap = smp.code_to_str(smp_code)
            data = smp_data

        elif l2cap_cid == l2cap.L2CAP_CID_LE_SCH:
            sch_code, sch_id, sch_length, sch_data = l2cap.parse_sch(l2cap_data)
            cmd_evt_l2cap = l2cap.sch_code_to_str(sch_code)
            data = sch_data
    else:
        raise Exception('Unknown HCI Packet Type')

    rows.append(row)

SyntaxError: invalid syntax (3457954134.py, line 29)

In [83]:
pprint(rows[0])

[{'packet_length': 47,
  'pkt_data': b'\x04>,\r\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00'
              b'\xff\x7f\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x12\x11\tJBL V'
              b'IBE BUDS-LE',
  'pkt_dst': 'Host',
  'pkt_src': 'Controller',
  'pkt_type': 'event',
  'sequence_number': 1},
 62,
 b'\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\xff\x7f\xc0\x00\x00\x00\x00\x00'
 b'\x00\x00\x00\x00\x12\x11\tJBL VIBE BUDS-LE',
 'EVENT LE_Meta_Event (0x3e)']


In [ ]:

if hci_type == hci_uart.HCI_CMD:
    opcode, length, data = hci_cmd.parse(data)
    cmd_evt_l2cap = hci_cmd.cmd_to_str(opcode)
    print(cmd_evt_l2cap)

In [33]:
cmd_evt_l2cap

NameError: name 'cmd_evt_l2cap' is not defined

[Reference](https://datatracker.ietf.org/doc/html/rfc1761)

Android bluetooth logs come in the **Snoop Version 1 Packet Capture File Format** which is similar to the second version developed by Sun Microsystems in 1995. When we capture logs of this format, we obtain arrays of octets (8 bit packets of information), with each array item corresponding to a packet record.
